# YAHOO電影爬蟲練習
## 這個作業主要是練習爬取YAHOO電影的資訊，主要會用到基本的GET method，以及解析伺服器回傳的html，從中擷取我們想要的資料。

### 這個範例示範如何爬取上映中的電影資訊。先訪問YAHOO電影首頁：https://movies.yahoo.com.tw ，點選「上映中」的分頁，先確認上映中的電影總共有幾筆，然後逐頁爬取。

In [1]:
import requests
import re
from bs4 import BeautifulSoup

In [2]:
# 先觀察一下目前上映中的電影數量
url = 'https://movies.yahoo.com.tw/movie_intheaters.html'
resp = requests.get(url)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
html = soup.find("div", attrs={'class':'release_box'})  # 尋找正在上映中的全部電影筆數，其所在的tag
print("正在上映中總共: ", html.p.string)

正在上映中總共:  共78筆，目前顯示1~10筆


### 爬取所有上映中的電影資料：電影名稱、期待度、滿意度、簡介等等。

In [3]:
# 接下來要爬取每一頁的電影介紹
# 先點選網頁最底部的下一頁，觀察其網址變化。然後我們把頁數當作變數傳入，用迴圈爬取。
max_page = 5
for page_number in range(1, max_page+1, 1):
    url = 'https://movies.yahoo.com.tw/movie_intheaters.html'
    payload = {'page':str(page_number)}
    resp = requests.get(url, params=payload)
    resp.encoding = 'utf-8'
    soup = BeautifulSoup(resp.text, 'lxml')

    # 把電影介紹擷取出來
    movie_list = soup.find("ul", attrs={"class":"release_list"}).find_all("li")
    for p in movie_list:
        print("----------------------------------------------------------------------")
        # 電影名稱
        movie_name = p.find("div", attrs={"class":"release_movie_name"})
        print("電影名稱：", movie_name.a.string)
        
        # 定位電影評價資訊欄位
        level_box = movie_name.find("dl", attrs={"class":"levelbox"})
        
        # 期待度
        expectation = level_box.find("div", attrs={"class":"leveltext"})
        print("期待度：", expectation.span.string)
        
        # 滿意度
        satisfaction = level_box.find_all("div", attrs={"class":"leveltext"})[1]
        print("滿意度：", satisfaction.span["data-num"])  # 滿意度可以從"data-num"這個屬性擷取
        
        # 簡介
        movie_info = p.find("div", attrs={"class":"release_text"})
        print(movie_info.span.string)


----------------------------------------------------------------------
電影名稱： 
                  去年聖誕節
期待度： 80%
滿意度： 3.7

                  一事無成的凱特（艾蜜莉亞克拉克 飾）在倫敦渾渾噩噩地過日子，她做出一連串的糟糕決定導致的厄運總是伴隨著鞋子上發出的鈴聲如影隨形地跟著她，因為她只能打扮成聖誕老公公的小精靈，在一間全年無休的聖誕飾品店當店員。於是當心地善良的大帥哥湯姆（亨利高汀 飾）闖入凱特的人生，並且開始幫助她克服她人生中的許多障礙與難關，這一切都似乎美好得有點不真實。當倫敦在聖誕佳節期間轉變成一座童話般的夢幻王國，這兩個看似八竿子打不著的人應該不會結成良緣，但有時候你得順其自然、聆聽內心的聲音…而且一定要擁有信念。
 
《去年聖誕節》一片中有許多喬治麥可的經典名曲，當然包括與電影同名，詞意苦中帶甜的經典聖誕歌曲。這部電影中也有一些這位曾經榮獲多項葛萊美獎的傳奇歌手從未發表過的全新歌曲。喬治麥可在輝煌的演藝生涯中，總共賣出1億1千5百萬張專輯，並且擁有10首冠軍單曲。
                
----------------------------------------------------------------------
電影名稱： 
                  呆萌特務
期待度： 76%
滿意度： 3.8

                  ★ 改編自德國格林童話「不來梅樂隊」，是部現代又歡樂的公路電影。 
★ 奧斯卡金像獎最佳動畫短片獎導演新作 
★ 角色設定逗趣、故事流暢 
★ 將動物擬人化，小人物的自我成長故事，適合大人小孩一起觀影 
★ IMDB 評分近 7 分，勝過《女王的柯基》 
 
小鎮盜竊案頻頻發生，一隻夢想成為大偵探的貓咪為了找出真凶決定離家出走。旅途中的她遇到了一隻自認風度翩翩的公雞，一隻夢想成為馬戲團明星的驢，一隻仇恨人類的守門狗。四隻小動物歷經磨合，即將找出真凶時，卻被誤認為是偷竊真凶。他們該如何證明清白，如何守護「友誼」和「夢想」…
                
----------------------------------------------------